In [32]:
import numpy as np
import pandas as pd
import urllib.request
import time
import requests
!conda install -c conda-forge folium  --yes
import folium #map rendering library
from bs4 import BeautifulSoup
from urllib.request import urlopen
!pip install geopy
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print("Imports done")

Solving environment: ...working... done

# All requested packages already installed.

Imports done


In [33]:
#Scraping Algiers_Province townships
url_city_1 = "https://en.wikipedia.org/wiki/Algiers_Province"
html_1=urlopen(url_city_1)
Soup_Algiers = BeautifulSoup(html_1,'html.parser')
Soup_Algiers= Soup_Algiers.find('div',attrs={'class' :'div-col'})

In [34]:
#A function that returns a list of townships ;
"""  args
  soup : BeuatifulsSoup Object
  return : list of Townships of type 'String'
"""
def Townships_list(soup):
  lis=soup.find_all('li')  
  data=[]
  for li in lis[0:]:
      result = li.text.strip().replace(u'\xa0', u'').replace(';','')
      data.append(result)
  return data

In [35]:
#creating a list of all townships in Algiers City (56)
Algiers_Townships=Townships_list(Soup_Algiers)
#creating dataframe of algiers townships from the list
Algiers=pd.DataFrame(Algiers_Townships,columns=['Township'])

In [42]:
Algiers_Townships

['Baba Ali[fr]',
 'Beau-Fraisier[fr]',
 'Casbah of Algiers',
 'Chevalley[ar]',
 'Cité 1er Novembre 1954[ar]',
 'Cité Yahia Boushaki',
 'Bachdjerrah',
 'Diar el Mahçoul',
 'El Annasser[it]',
 'El Hamiz[fr]',
 'Birkhadem',
 'La Glacière[fr]',
 'Bologhine',
 'Les Fusillés[fr]',
 'Moretti[fr]',
 'Riadh El Feth[ar]',
 'Saïd Hamdine[fr]',
 'Sidi Abdellah[fr]',
 'Sidi Yahia[fr]',
 'Tamentfoust']

In [36]:
#Manually replacing some names of the townships to match the names with the geopy nominatim package
Algiers_Townships[6]= 'Bachdjerrah'
Algiers_Townships[10]='Birkhadem'
Algiers_Townships[12]='Bologhine'

In [37]:
#Creating a function that returns a dataframe that contains a townships' coordinates 

"""
args : 
towns_list : a list of townships names of string format
City : The name of the City 


return :City_coordinates :a pandas dataframe of cities coordinates 
"""
def coordinates_city(towns_list,City):
   lat=[]
   lon=[]
   for town in towns_list:
        geolocator = Nominatim(user_agent="Explorer")
        location = geolocator.geocode(town+','+City)
        if location:
          latitude = location.latitude
          longitude = location.longitude
        else:
          latitude='None'
          longitude='None'
        lat.append(latitude)
        lon.append(longitude)
   City_coordinates=pd.DataFrame({'Latitude':lat,'Longitude':lon})
   return City_coordinates

In [38]:
#Getting the Latitude and longitude of the townships one by one
Algiers_city=coordinates_city(Algiers_Townships,'Algiers')
#Assigning the relevant names from the townships Dataframe
Algiers_city['Township']=Algiers['Township']

In [41]:
print(Algiers_city)

   Latitude Longitude                    Township
0      None      None                Baba Ali[fr]
1      None      None           Beau-Fraisier[fr]
2   36.7847   3.05959           Casbah of Algiers
3      None      None               Chevalley[ar]
4   36.6866   3.21761  Cité 1er Novembre 1954[ar]
5      None      None         Cité Yahia Boushaki
6    36.723   3.11307           Club des Pins[ar]
7   36.7444   3.06842             Diar el Mahçoul
8      None      None             El Annasser[it]
9      None      None                El Hamiz[fr]
10  36.7155   3.04907                   Hamma[fr]
11     None      None             La Glacière[fr]
12  36.8047   3.04262               Les Dunes[fr]
13     None      None            Les Fusillés[fr]
14     None      None                 Moretti[fr]
15  36.1631   5.71319           Riadh El Feth[ar]
16     None      None            Saïd Hamdine[fr]
17     None      None           Sidi Abdellah[fr]
18     None      None              Sidi Yahia[fr]


In [39]:
# create a folium marked map for a city 

"""
args :
  City_df : a dataframe that contains the township's names , Latitudes and Longitudes
  City_name: The relevant city of the townships
return: A displayed marked folium map 


"""
def folium_marked_map(City_df,City_name):
  geolocator = Nominatim(user_agent="Explorer")
  location = geolocator.geocode(City_name)
  map_city = folium.Map(location=[location.latitude,location.longitude], zoom_start=10)
  # add markers to map
  for lt, lng, town in zip(City_df['Latitude'], City_df['Longitude'], City_df['Township']):
    label = town
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
      [lt, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7,
      parse_html=False).add_to(map_city)
  return map_city

In [40]:
#Create a folium Marked map of a city('Algiers')
Algiers_map= folium_marked_map(Algiers_city,'Algiers')

ValueError: Location should consist of two numerical values, but 'None' of type <class 'str'> is not convertible to float.